In [3]:
import numpy as np
import torch
import torch.utils.data
import torch.nn as nn
import torch.optim as optim

import torchdiffeq

from tensorboard_utils import Tensorboard
from tensorboard_utils import tensorboard_event_accumulator

import transformer.Constants as Constants
from transformer.Layers import EncoderLayer, DecoderLayer
from transformer.Modules import ScaledDotProductAttention
from transformer.Models import Decoder, get_attn_key_pad_mask, get_non_pad_mask, get_sinusoid_encoding_table
from transformer.SubLayers import PositionwiseFeedForward

import dataset

import model_process
import checkpoints
from node_transformer import NodeTransformer

import matplotlib
import numpy as np
import matplotlib.pyplot as plt
#%matplotlib notebook  
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

print("Torch Version", torch.__version__)

%load_ext autoreload
%autoreload 2

Torch Version 1.1.0
The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [4]:
seed = 1
torch.manual_seed(seed)
device = torch.device("cuda")
print("device", device)

device cuda


In [5]:
data = torch.load("/home/mandubian/datasets/multi30k/multi30k.atok.low.pt")

In [6]:
max_token_seq_len = data['settings'].max_token_seq_len
print(max_token_seq_len)

52


In [7]:
train_loader, val_loader = dataset.prepare_dataloaders(data, batch_size=128)

### Create an experiment with a name and a unique ID

In [8]:
exp_name = "node_transformer_adams_multi30k"
unique_id = "2019-06-10_1000"

# unique_id = "2019-06-10_1000"
# default + adams
# batch 128
# rtol=1e-2, atol=1e-2
# lr=1e-5


### Create Model

In [9]:
model = None

In [14]:
from odeint_ext_adams import *

src_vocab_sz = train_loader.dataset.src_vocab_size
print("src_vocab_sz", src_vocab_sz)
tgt_vocab_sz = train_loader.dataset.tgt_vocab_size
print("tgt_vocab_sz", tgt_vocab_sz)

if model:
    del model

model = NodeTransformer(
    n_src_vocab=max(src_vocab_sz, tgt_vocab_sz),
    n_tgt_vocab=max(src_vocab_sz, tgt_vocab_sz),
    len_max_seq=max_token_seq_len,
    #emb_src_tgt_weight_sharing=False,
    #d_word_vec=64, d_model=64, d_inner=256,
    n_head=8, method='adams-ext', rtol=1e-2, atol=1e-2,
    has_node_encoder=True, has_node_decoder=True)

model = model.to(device)

src_vocab_sz 9795
tgt_vocab_sz 17989


### Create Tensorboard metrics logger

In [15]:
tb = Tensorboard(exp_name, unique_name=unique_id)

Writing TensorBoard events locally to runs/node_transformer_adams_multi30k_2019-06-10_1000


### Create basic optimizer

In [16]:
optimizer = optim.Adam(model.parameters(), lr=1e-4, betas=(0.9, 0.995), eps=1e-9)


### Train

In [20]:
# Continuous space discretization
timesteps = np.linspace(0., 1, num=6)
timesteps = torch.from_numpy(timesteps).float()

EPOCHS = 50
LOG_INTERVAL = 5

#from torch import autograd
#with autograd.detect_anomaly():
model_process.train(
    exp_name, unique_id,
    model, 
    train_loader, val_loader, timesteps,
    optimizer, device,
    epochs=EPOCHS, tb=tb, log_interval=LOG_INTERVAL,
    start_epoch=0 #, best_valid_accu=state["acc"]
)

Loaded model and timesteps to cuda


[ Epoch 0 ]
Adding group train to writers (dict_keys([]))
[Training]  loss: 13.472290847040703, ppl:  709482.31342, accuracy: 12.995 %, elapse: 927099.388ms
Adding group eval to writers (dict_keys(['train']))
[Validation]  loss: 10.20736033875994,  ppl:  27101.93316, accuracy: 24.952 %, elapse: 3534.781ms
Checkpointing Validation Model...


[ Epoch 1 ]


KeyboardInterrupt: 

In [ ]:
# Continuous space discretization
timesteps = np.linspace(0., 1, num=12)
timesteps = torch.from_numpy(timesteps).float()

EPOCHS = 50
LOG_INTERVAL = 5

#from torch import autograd
#with autograd.detect_anomaly():
model_process.train(
    exp_name, unique_id,
    model, 
    train_loader, val_loader, timesteps,
    optimizer, device,
    epochs=EPOCHS, tb=tb, log_interval=LOG_INTERVAL,
    start_epoch=21, best_valid_accu=state["acc"]
)

Loaded model and timesteps to cuda


[ Epoch 21 ]
[Training]  loss: 5.982365558516917, ppl:  396.37691, accuracy: 41.378 %, elapse: 2633471.951ms
[Validation]  loss: 5.288381585692922,  ppl:  198.02268, accuracy: 43.463 %, elapse: 2984.516ms


[ Epoch 22 ]


### Restore best checkpoint (to restart past training)

In [14]:
state = checkpoints.restore_best_checkpoint(
    exp_name, unique_id, "validation", model, optimizer)

print("accuracy", state["acc"])
print("loss", state["loss"])
model = model.to(device)

Extracting state from checkpoints/node_transformer_full_multi30k_2019-06-08_1300_validation_best.pth
Loading model state_dict from state found in checkpoints/node_transformer_full_multi30k_2019-06-08_1300_validation_best.pth
Loading optimizer state_dict from state found in checkpoints/node_transformer_full_multi30k_2019-06-08_1300_validation_best.pth
accuracy 0.4476681710724264
loss 5.363942295853034
